# Reformat data to SQL

In [7]:
import pandas
import csv


def load_data(filename):
    data = pandas.read_csv(filename)
    if "STATEFP" in data.columns and "COUNTYFP" in data.columns:
        data = data.rename(columns={"STATEFP": "state_id", "COUNTYFP": "county_id"})
        data = data.melt(id_vars=["state_id", "county_id"])
    else:
        data = data.rename(columns={"id": "country_id"})
    data = data[data["value"].notnull()]
    return data


def save_data(data, filename):
    ids = []
    if "state_id" in data.columns and "county_id" in data.columns:
        ids = ["state_id", "county_id"]
    else:
        ids = ["country_id"]
    data = data[
        ids
        + [
            "value",
            "start_date",
            "end_date",
            "source",
            "dataset",
        ]
    ]
    data.to_csv(
        filename,
        index=False,
        quoting=csv.QUOTE_NONNUMERIC,
        quotechar="'",
        line_terminator="),\n(",
    )
    return data

# People experiencing homelessness

In [3]:
data = load_data("../homelessness/output/homelessness.csv")
data["start_date"] = "2019-01-01"
data["end_date"] = "2019-12-31"
data["source"] = 17 # Housing and Urban Development
data["dataset"] = 92 # homeless_per_10000
data = save_data(data, "output/homeless_reformatted.csv")
data

,state_id,county_id,value,start_date,end_date,source,dataset
0,2,13,26.07,2019-01-01,2019-12-31,17,92
1,2,16,26.07,2019-01-01,2019-12-31,17,92
2,2,20,26.07,2019-01-01,2019-12-31,17,92
3,2,50,26.07,2019-01-01,2019-12-31,17,92
4,2,60,26.07,2019-01-01,2019-12-31,17,92
...,...,...,...,...,...,...,...
3215,56,37,9.47,2019-01-01,2019-12-31,17,92
3216,56,39,9.47,2019-01-01,2019-12-31,17,92
3217,56,41,9.47,2019-01-01,2019-12-31,17,92
3218,56,43,9.47,2019-01-01,2019-12-31,17,92


## Mortality

In [4]:
metadata = pandas.DataFrame(
    data={
        "variable": [
            "deaths_0-5_all",
            "deaths_5-25_all",
            "deaths_25+_all",
            "deaths_25+_circ",
            "deaths_25+_resp",
            "mort_0-5_all",
            "mort_0-5_all_est",
            "mort_5-25_all",
            "mort_5-25_all_est",
            "mort_25+_all",
            "mort_25+_all_est",
            "mort_25+_circ",
            "mort_25+_circ_est",
            "mort_25+_resp",
            "mort_25+_resp_est",
        ],
        "dataset": [
            70,
            71,
            72,
            73,
            74,
            75,
            76,
            77,
            78,
            79,
            80,
            81,
            82,
            83,
            84,
        ],
    }
)

data = load_data("input/mortality.csv")
data["source"] = 13  # cdc
data["start_date"] = "2016-01-01"
data["end_date"] = "2016-12-31"
data = data = data.merge(metadata)
data = save_data(data, "output/mortality_reformatted.csv")
data

,state_id,county_id,value,start_date,end_date,source,dataset
0,1,1,157.00,2016-01-01,2016-12-31,13,73
1,1,3,631.00,2016-01-01,2016-12-31,13,73
2,1,5,81.00,2016-01-01,2016-12-31,13,73
3,1,7,73.00,2016-01-01,2016-12-31,13,73
4,1,9,184.00,2016-01-01,2016-12-31,13,73
...,...,...,...,...,...,...,...
32605,56,37,0.45,2016-01-01,2016-12-31,13,78
32606,56,39,0.45,2016-01-01,2016-12-31,13,78
32607,56,41,0.45,2016-01-01,2016-12-31,13,78
32608,56,43,0.45,2016-01-01,2016-12-31,13,78


## PM 2.5

In [5]:
data = load_data("input/climate.csv")
data = data[data["variable"] == "PM2_5"]
data["start_date"] = "2015-01-01"
data["end_date"] = "2015-12-31"
data["source"] = 12 # NASA Earth
data["dataset"] = 69 # PM2_5
data = save_data(data, "output/climate_reformatted.csv")
data

,state_id,county_id,value,start_date,end_date,source,dataset
302680,29,227,7.86,2015-01-01,2015-12-31,12,69
302681,31,61,6.26,2015-01-01,2015-12-31,12,69
302682,36,13,7.82,2015-01-01,2015-12-31,12,69
302683,37,181,8.84,2015-01-01,2015-12-31,12,69
302684,47,183,8.30,2015-01-01,2015-12-31,12,69
...,...,...,...,...,...,...,...
305817,40,91,6.82,2015-01-01,2015-12-31,12,69
305818,40,95,6.35,2015-01-01,2015-12-31,12,69
305819,48,225,9.38,2015-01-01,2015-12-31,12,69
305820,48,407,8.39,2015-01-01,2015-12-31,12,69


## Energy Expenditure

In [6]:
metadata = pandas.DataFrame(
    data={
        "variable": [
            "energy_expenditure_per_capita",
            "transportation_energy_expenditure_per_capita",
            "residential_energy_expenditure_per_capita",
            "energy_expenditure_share_of_gdp",
            "transportation_energy_expenditure_share_of_gdp",
            "residential_energy_expenditure_share_of_gdp",
        ],
        "dataset": [
            85,
            87,
            86,
            88,
            90,
            89,
        ],
    }
)
data = load_data("input/energy.csv")
data = data.merge(metadata)
data = data.drop(columns=["variable"])
data["start_date"] = "2018-01-01"
data["end_date"] = "2018-12-31"
data["source"] = 14  # U.S. Energy Information Administration
data = save_data(data, "output/energy_reformatted.csv")
data


,state_id,county_id,value,start_date,end_date,source,dataset
0,2,13,7793.051771,2018-01-01,2018-12-31,14,85
1,2,16,7793.051771,2018-01-01,2018-12-31,14,85
2,2,20,7793.051771,2018-01-01,2018-12-31,14,85
3,2,50,7793.051771,2018-01-01,2018-12-31,14,85
4,2,60,7793.051771,2018-01-01,2018-12-31,14,85
...,...,...,...,...,...,...,...
18847,56,37,0.013075,2018-01-01,2018-12-31,14,89
18848,56,39,0.013075,2018-01-01,2018-12-31,14,89
18849,56,41,0.013075,2018-01-01,2018-12-31,14,89
18850,56,43,0.013075,2018-01-01,2018-12-31,14,89


## Critical Habitat

In [7]:
data = load_data("input/climate.csv")
data = data[data["variable"] == "PerCritHab"]
data["start_date"] = "2021-01-01"
data["end_date"] = "2021-12-31"
data["source"] = 16 # fish and wildlife service
data["dataset"] = 91 # critical_habitat
data = save_data(data, "output/critical_habitat_reformatted.csv")
data

,state_id,county_id,value,start_date,end_date,source,dataset
299460,29,227,0.0,2021-01-01,2021-12-31,16,91
299461,31,61,0.0,2021-01-01,2021-12-31,16,91
299462,36,13,0.0,2021-01-01,2021-12-31,16,91
299463,37,181,0.0,2021-01-01,2021-12-31,16,91
299464,47,183,0.0,2021-01-01,2021-12-31,16,91
...,...,...,...,...,...,...,...
302597,40,91,0.0,2021-01-01,2021-12-31,16,91
302598,40,95,0.0,2021-01-01,2021-12-31,16,91
302599,48,225,0.0,2021-01-01,2021-12-31,16,91
302600,48,407,0.0,2021-01-01,2021-12-31,16,91


# Population (US Census 5 year ACS, 2019)

In [8]:
data = load_data("input/population_2019.csv")
data["start_date"] = "2019-01-01"
data["end_date"] = "2019-12-31"
data["source"] = 6 # US Census Bureau
data["dataset"] = 93 # population
data = save_data(data, "output/population_reformatted.csv")
data

,state_id,county_id,value,start_date,end_date,source,dataset
0,17,51,7737.0,2019-01-01,2019-12-31,6,93
1,17,107,10797.0,2019-01-01,2019-12-31,6,93
2,17,165,9972.0,2019-01-01,2019-12-31,6,93
3,17,97,246122.0,2019-01-01,2019-12-31,6,93
4,17,127,5822.0,2019-01-01,2019-12-31,6,93
...,...,...,...,...,...,...,...
2899,47,33,5491.0,2019-01-01,2019-12-31,6,93
2900,47,95,2243.0,2019-01-01,2019-12-31,6,93
2901,47,93,187319.0,2019-01-01,2019-12-31,6,93
2902,53,5,72121.0,2019-01-01,2019-12-31,6,93


# Population (US Census Vintage 2019 estimates)

In [15]:
data = load_data("input/Population.csv")
data["start_date"] = "2019-01-01"
data["end_date"] = "2019-12-31"
data["source"] = 6 # US Census Bureau
data["dataset"] = 93 # population
data = data[data.variable == 'POPESTIMATE2019']
data = save_data(data, "output/population_all_2019_reformatted.csv")
data

,state_id,county_id,value,start_date,end_date,source,dataset
31420,1,1,55869.0,2019-01-01,2019-12-31,6,93
31421,1,3,223234.0,2019-01-01,2019-12-31,6,93
31422,1,5,24686.0,2019-01-01,2019-12-31,6,93
31423,1,7,22394.0,2019-01-01,2019-12-31,6,93
31424,1,9,57826.0,2019-01-01,2019-12-31,6,93
...,...,...,...,...,...,...,...
34557,56,37,42343.0,2019-01-01,2019-12-31,6,93
34558,56,39,23464.0,2019-01-01,2019-12-31,6,93
34559,56,41,20226.0,2019-01-01,2019-12-31,6,93
34560,56,43,7805.0,2019-01-01,2019-12-31,6,93


# Endangered Species

In [10]:
data = load_data("input/endspecies_total.csv")
data = data[data["variable"] == "Count_"]
data["start_date"] = "2018-01-01"
data["end_date"] = "2018-12-31"
data["source"] = 18 # Global Biodiversity Information Facility
data["dataset"] = 94 # endangered_species
data = save_data(data, "output/endangered_species_reformatted.csv")
data

,state_id,county_id,value,start_date,end_date,source,dataset
22540,29,227,0,2018-01-01,2018-12-31,18,94
22541,31,61,0,2018-01-01,2018-12-31,18,94
22542,36,13,133,2018-01-01,2018-12-31,18,94
22543,37,181,1,2018-01-01,2018-12-31,18,94
22544,47,183,1,2018-01-01,2018-12-31,18,94
...,...,...,...,...,...,...,...
25755,72,123,0,2018-01-01,2018-12-31,18,94
25756,72,97,0,2018-01-01,2018-12-31,18,94
25757,72,53,0,2018-01-01,2018-12-31,18,94
25758,72,121,0,2018-01-01,2018-12-31,18,94


# Wildfire Risk

In [11]:
data = load_data("input/Wildfire Risk.csv")
data = data[data.variable == 'Mean BP']
data.state_id = data.state_id.astype(int)
data.county_id = data.county_id.astype(int)
data["source"] = 19 # USDA Forest Service
data["start_date"] = "2021-01-01"
data["end_date"] = "2021-12-31"
data["dataset"] = 95 # wildfire_risk
data = save_data(data, "output/wildfire_risk_reformatted.csv")
data

,state_id,county_id,value,start_date,end_date,source,dataset
21994,1,1,0.000112,2021-01-01,2021-12-31,19,95
21995,1,3,0.001012,2021-01-01,2021-12-31,19,95
21996,1,5,0.000129,2021-01-01,2021-12-31,19,95
21997,1,7,0.000221,2021-01-01,2021-12-31,19,95
21998,1,9,0.00031,2021-01-01,2021-12-31,19,95
...,...,...,...,...,...,...,...
25131,56,37,0.000114,2021-01-01,2021-12-31,19,95
25132,56,39,0.001262,2021-01-01,2021-12-31,19,95
25133,56,41,0.000797,2021-01-01,2021-12-31,19,95
25134,56,43,0.000206,2021-01-01,2021-12-31,19,95


# Biodiversity

In [10]:
data = load_data("../biodiversity/csv/RLI.csv")
data["source"] = 20 # red list index
data["dataset"] = 96 # red_list_index
data["start_date"] = data["year"].apply(lambda x: f"{x}-01-01")
data["end_date"] = data["year"].apply(lambda x: f"{x}-12-31")
data = save_data(data, "output/red_list_index_reformatted.csv")
data


,country_id,value,start_date,end_date,source,dataset
0,4,0.83783,2000-01-01,2000-12-31,20,96
1,8,0.88873,2000-01-01,2000-12-31,20,96
2,12,0.91134,2000-01-01,2000-12-31,20,96
3,24,0.92899,2000-01-01,2000-12-31,20,96
4,32,0.84844,2000-01-01,2000-12-31,20,96
...,...,...,...,...,...,...
3951,704,0.71067,2022-01-01,2022-12-31,20,96
3952,732,0.93477,2022-01-01,2022-12-31,20,96
3953,887,0.83381,2022-01-01,2022-12-31,20,96
3954,894,0.87625,2022-01-01,2022-12-31,20,96
